In [1]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp
from sklearn.metrics import confusion_matrix

In [15]:
def sum_on_samples(R, samples, n_classes):
    G = []
    for j in range(n_classes):
        class_sum = 0
        for i in samples:
            class_sum += R[(i, j)]
        G.append(class_sum)
    return G


def sum_on_classes(R, samples, n_classes):
    S = []
    for i in samples:
        sample_sum = 0
        for j in range(n_classes):
            sample_sum += R[(i, j)]
        S.append(sample_sum)
    return S


def get_constraint_for_classes(global_constraint, n_samples):
    n = np.ones(len(global_constraint))
    for j, const_num in enumerate(global_constraint):
        if const_num == -1:
            n[j] = 1.0
        else:
            n[j] = const_num / n_samples  # percentage for each constraint
    return n


def operation_research_func(samples, cost_matrix, global_constarint, subgroup_info, random_state =42):
    np.random.seed(random_state)
    R = {}
    objective_function = 0
    n_samples = samples.shape[0]
    predict_labels_proba = samples.drop('subgroup', axis=1)
    samples_idx = samples.index
    n_classes = predict_labels_proba.shape[1] 
    predict_hard = np.zeros(n_samples).astype(int)
    
    solver = pywraplp.Solver.CreateSolver('GLOP')
    if len(cost_matrix.shape)<=2:
        cost_dot_proba = np.dot(cost_matrix, predict_labels_proba.transpose())
    else:
        cost_dot_proba  = []
        for i in range(cost_matrix.shape[0]):
            cost_dot_proba.append(np.dot(cost_matrix[i], predict_labels_proba.loc[i]))
        cost_dot_proba = np.array(cost_dot_proba).transpose()

    # create variables 
    for i in range(n_samples):
        for j in range(n_classes):
            R[(i, j)] = solver.NumVar(0, solver.infinity(), 'R{}{}'.format(i, j))
            objective_function += R[(i, j)] * cost_dot_proba[j, i] 
    
    # Objective function
    solver.Minimize(objective_function)
    
    print("Number of variables =", solver.NumVariables())

    # Constraints
    
    #global constraint
    n = get_constraint_for_classes(global_constarint, n_samples) # constraint for each class
    G = sum_on_samples(R, samples_idx, n_classes) 
    for i in range(n_classes):
        solver.Add(G[i] <= np.ceil(n_samples * n[i])) 
    
    #each sample for one class
    S = sum_on_classes(R, samples_idx, n_classes) 
    for j in range(n_samples):
        solver.Add(S[j] >= 1)
        
    #local constraint
    for k, value in subgroup_info.items():
        samples_subgroup = samples[samples['subgroup'] == k]
        # n = get_constraint_for_classes(n_classes, value['const_num'], value['constrained_classes'], samples_subgroup.shape[0])
        n = get_constraint_for_classes(value, samples_subgroup.shape[0])
        L = sum_on_samples(R, samples_subgroup.index, n_classes)
        for i in range(n_classes):
            solver.Add(L[i] <= np.ceil(samples_subgroup.shape[0] * n[i])) 
    # 
    print('Number of constraints =', solver.NumConstraints())

    status = solver.Solve()

    for i in range(n_samples):
        for j in range(n_classes):
            print(' R[({}, {})] ='.format(i, j),  R[(i, j)].solution_value())
            if R[(i, j)].solution_value() == 1:
                predict_hard[i] = j # the sample get the class
            if R[(i, j)].solution_value() not in [0,1]:
                print('The solution is not 0 or 1')

        
    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value():0.1f}")

    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")
    
    return solver.Objective().Value(), predict_hard

In [3]:
# cost_matrix = np.array([[[0,10],[1,1]], [[0,10],[1,1]], [[0,10],[2,1]], [[0,0],[1,1]]])
# cost_matrix.shape

In [4]:
# #small example
# #predict_labels_proba = np.array([[0.9,0.1], [0.7,0.3], [0.3,0.7], [0.2, 0.8]])#, [0.2 ,0.8], [0.3, 0.7]])
# predict_labels_proba = pd.DataFrame([[0.9,0.1], [0.7,0.3], [0.3,0.7], [0.2, 0.8]])
# samples = predict_labels_proba
# sub1 = '867c4a38-0d75-46ff-a4c2-f44cb63be2c9'
# sub2 = '280a19b6-a767-4e7f-b6b4-e422d6876897'
# samples['subgroup'] = [sub1, sub2, sub1, sub2]
# #cost_matrix = np.array([[[0,10],[1,1]], [[0,10],[1,1]], [[0,10],[2,1]], [[0,0],[1,1]]])
# cost_matrix = np.array([[0,10],[1,1]])
# # const_num = 1
# # constrained_classes = [1]
# global_constraint = [-1, 3]
# #subsets = {0: {'samples': [0, 2], 'constrained_classes': [1], 'const_num': 2} , 1: {'samples': [1, 3], 'constrained_classes': [1], 'const_num': 2}}
# #subgroup_info = {0: { 'constrained_classes': [1], 'const_num': 2} , 1: {'constrained_classes': [1], 'const_num': 2}}
# subgroup_info = {sub1: [-1, 2] , sub2: [-1, 2]}

In [20]:
samples_all = pd.read_csv('data/predict_proba.csv', index_col=False)
samples_all = samples_all.rename(columns={'provider_id': 'subgroup', '0': 0, '1': 1})
samples_all

,0,1,subgroup
0,0.531627,0.468373,867c4a38-0d75-46ff-a4c2-f44cb63be2c9
1,0.108769,0.891231,280a19b6-a767-4e7f-b6b4-e422d6876897
2,0.365181,0.634819,89e6e230-1333-448b-9800-dc2d09a85e99
3,0.984345,0.015655,63260486-7618-4660-9445-7e4885e99042
4,0.782031,0.217969,50761787-31da-4b17-8048-c6063b614b80
...,...,...,...
4995,0.885391,0.114609,d3c35a47-5967-4bef-a39b-feafed32261c
4996,0.531627,0.468373,451e185d-5ad1-469f-8c33-164884140ffc
4997,0.108769,0.891231,867c4a38-0d75-46ff-a4c2-f44cb63be2c9
4998,0.108769,0.891231,75f8d952-be81-4e22-951e-41b291d211f2


The constraints (global and local) are defined as a list with a constraint for each class (ordered). The constraint is the total number of samples for each class and -1 is for classes that don't have constraints. 

In [21]:
constraint_precent = 0.4
subgroup_constraint = {}
for provider, count in samples_all['subgroup'].value_counts().items():
    subgroup_constraint[provider] = [-1, np.ceil(constraint_precent*count)]
subgroup_constraint

{'867c4a38-0d75-46ff-a4c2-f44cb63be2c9': [-1, 309.0],
 'a12f6956-9aca-4550-8caa-2e2f9532674c': [-1, 125.0],
 '02531004-4599-4177-9d7d-00bc85a200c6': [-1, 100.0],
 '856f211d-c66e-4db7-b910-7419900a70e1': [-1, 81.0],
 '75f8d952-be81-4e22-951e-41b291d211f2': [-1, 73.0],
 '451e185d-5ad1-469f-8c33-164884140ffc': [-1, 43.0],
 'c8fa65fe-4bf6-4846-aaaa-62f8937e31bf': [-1, 42.0],
 '22f309a8-1746-49a7-bf6c-678729834573': [-1, 38.0],
 'd712a1f3-812a-4f41-a567-07b97ce32fa1': [-1, 29.0],
 'a0dc02cc-c56b-4931-ad79-8ed0d3bd2ffc': [-1, 29.0],
 '444b0293-8056-47e0-a3c8-cd7bca4997d3': [-1, 28.0],
 'a68c8f2a-f1a7-4c64-9dc4-d641c9371725': [-1, 28.0],
 '172d3f92-3da0-4ad9-b8e8-12d95102b35f': [-1, 23.0],
 'b954c933-d771-4dc0-8f4f-89db56cf7266': [-1, 22.0],
 '318dfa35-b3d4-48ac-83b0-a4428d01444c': [-1, 22.0],
 '555339a4-dbff-43f2-9c92-b0e03b1c6312': [-1, 20.0],
 '3f33a0b8-d4da-445a-baa3-678dfb0d8763': [-1, 18.0],
 '148d76c1-3e78-4581-b909-e4abd3496e12': [-1, 18.0],
 'dc4abe91-b03f-4dd2-9987-102db5767235': [-

In [29]:
user_1_cost_matrix = np.array([[0,10],[1,1]])
user_2_cost_matrix = np.array([[0,20],[1,1]])
user_3_cost_matrix = np.array([[0,30],[1,1]])

In [30]:
user_3_cost_matrix

array([[ 0, 30],
       [ 1,  1]])

In [31]:
np.random.seed(33)
users_split = np.random.randint(3, size=samples_all.shape[0])

In [32]:
cost_matrix = []
for i in users_split:
    if i == 0:
        cost_matrix.append(user_1_cost_matrix)
    elif i == 1:
        cost_matrix.append(user_2_cost_matrix)
    else:
        cost_matrix.append(user_3_cost_matrix)
cost_matrix = np.array(cost_matrix)
cost_matrix.shape

(5000, 2, 2)

In [33]:
#cost_matrix = np.array([[0,1],[1,0]])
global_constraint = [-1, 500]

In [34]:
objective_function_value, predict_hard= operation_research_func(samples_all, cost_matrix, global_constraint, subgroup_constraint, random_state =42)

Number of variables = 10000
Number of constraints = 6782
 R[(0, 0)] = 1.0
 R[(0, 1)] = 0.0
 R[(1, 0)] = 1.0
 R[(1, 1)] = 0.0
 R[(2, 0)] = 0.0
 R[(2, 1)] = 1.0
 R[(3, 0)] = 1.0
 R[(3, 1)] = 0.0
 R[(4, 0)] = 1.0
 R[(4, 1)] = 0.0
 R[(5, 0)] = 1.0
 R[(5, 1)] = 0.0
 R[(6, 0)] = 1.0
 R[(6, 1)] = 0.0
 R[(7, 0)] = 1.0
 R[(7, 1)] = 0.0
 R[(8, 0)] = 1.0
 R[(8, 1)] = 0.0
 R[(9, 0)] = 1.0
 R[(9, 1)] = 0.0
 R[(10, 0)] = 1.0
 R[(10, 1)] = 0.0
 R[(11, 0)] = 1.0
 R[(11, 1)] = 0.0
 R[(12, 0)] = 1.0
 R[(12, 1)] = 0.0
 R[(13, 0)] = 1.0
 R[(13, 1)] = 0.0
 R[(14, 0)] = 1.0
 R[(14, 1)] = 0.0
 R[(15, 0)] = 1.0
 R[(15, 1)] = 0.0
 R[(16, 0)] = 1.0
 R[(16, 1)] = 0.0
 R[(17, 0)] = 1.0
 R[(17, 1)] = 0.0
 R[(18, 0)] = 1.0
 R[(18, 1)] = 0.0
 R[(19, 0)] = 1.0
 R[(19, 1)] = 0.0
 R[(20, 0)] = 1.0
 R[(20, 1)] = 0.0
 R[(21, 0)] = 0.0
 R[(21, 1)] = 1.0
 R[(22, 0)] = 1.0
 R[(22, 1)] = 0.0
 R[(23, 0)] = 1.0
 R[(23, 1)] = 0.0
 R[(24, 0)] = 1.0
 R[(24, 1)] = 0.0
 R[(25, 0)] = 1.0
 R[(25, 1)] = 0.0
 R[(26, 0)] = 1.0
 R[(26, 1

In [35]:
sum(predict_hard)

500

In [14]:
samples_all['predict_hard_constraint'] = predict_hard
samples_all

,0,1,subgroup,predict_hard_constraint
0,0.531627,0.468373,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0
1,0.108769,0.891231,280a19b6-a767-4e7f-b6b4-e422d6876897,0
2,0.365181,0.634819,89e6e230-1333-448b-9800-dc2d09a85e99,0
3,0.984345,0.015655,63260486-7618-4660-9445-7e4885e99042,0
4,0.782031,0.217969,50761787-31da-4b17-8048-c6063b614b80,0
...,...,...,...,...
4995,0.885391,0.114609,d3c35a47-5967-4bef-a39b-feafed32261c,0
4996,0.531627,0.468373,451e185d-5ad1-469f-8c33-164884140ffc,0
4997,0.108769,0.891231,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0
4998,0.108769,0.891231,75f8d952-be81-4e22-951e-41b291d211f2,0


In [36]:
samples_all = samples_all.rename(columns={ '0': 0, '1': 1})
samples_all = samples_all.drop('predict_hard_max', axis=1)

In [51]:
samples_all['predict_hard_max'] = samples_all[[0, 1]].idxmax(axis="columns").astype(int)
samples_all

,0,1,subgroup,predict_hard_constraint,predict_hard_max
0,0.531627,0.468373,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,0,0
1,0.108769,0.891231,280a19b6-a767-4e7f-b6b4-e422d6876897,1,1
2,0.365181,0.634819,89e6e230-1333-448b-9800-dc2d09a85e99,0,1
3,0.984345,0.015655,63260486-7618-4660-9445-7e4885e99042,0,0
4,0.782031,0.217969,50761787-31da-4b17-8048-c6063b614b80,0,0
...,...,...,...,...,...
4995,0.885391,0.114609,d3c35a47-5967-4bef-a39b-feafed32261c,0,0
4996,0.531627,0.468373,451e185d-5ad1-469f-8c33-164884140ffc,0,0
4997,0.108769,0.891231,867c4a38-0d75-46ff-a4c2-f44cb63be2c9,1,1
4998,0.108769,0.891231,75f8d952-be81-4e22-951e-41b291d211f2,1,1


In [46]:
y_pred = np.array(samples_all['predict_hard_constraint'])
y_pred

array([0, 1, 0, ..., 1, 1, 1])

In [52]:
y_true = np.array(samples_all['predict_hard_max'])
y_true

array([0, 1, 1, ..., 1, 1, 1])

In [53]:
confusion_matrix(y_true, y_pred)

array([[2736,    0],
       [1764,  500]], dtype=int64)

In [56]:
subgroup_prediction = {}
for provider in samples_all['subgroup']:
    subgroup_prediction[provider] = count
subgroup_prediction

{0: 4500, 1: 500}

In [68]:
subgroup_info = samples_all.groupby(['subgroup']).sum()[['predict_hard_constraint', 'predict_hard_max']]
subgroup_info['n_sample'] = samples_all.groupby(['subgroup']).count()[0]
subgroup_info['constraint'] = np.ceil(samples_all.groupby(['subgroup']).count()[0]*constraint_precent)

In [71]:
subgroup_info

,predict_hard_constraint,predict_hard_max,n_sample,constraint
subgroup,,,,
00c7cc34-c4a8-432c-9a1d-53e54b3da2a9,0,0,6,3.0
00ff5627-038f-4a69-9ab2-3bbebb927042,0,1,1,1.0
0156d05a-906b-43fb-bef9-600f8d49c740,0,0,1,1.0
020111e3-2aa1-43c6-b789-4f9e4a395546,0,1,1,1.0
02519b96-f37e-4db5-8593-40a913fc554f,0,0,1,1.0
...,...,...,...,...
fe91abcc-a2c4-4b78-b588-b6cb6aa861a7,0,0,1,1.0
fec69360-bc66-4e53-9194-4f771d3a929f,0,1,1,1.0
fef4407e-10a3-4898-8b4f-7e1b577ac4b3,0,0,1,1.0


In [70]:
subgroup_info['predict_hard_max'].sum()

2264